In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

print(tf.__version__)

2.7.0


In [2]:
dataset = pd.read_csv('./data/Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values
print(X)
print(y)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
[1 0 1 ... 1 1 0]


In [3]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[1.0 0.0 1.0 ... 1 1 101348.88]
 [1.0 0.0 0.0 ... 0 1 112542.58]
 [1.0 0.0 1.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 1.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 1.0 ... 1 0 38190.78]]


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [9]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 3s 5ms/step - loss: 0.5659 - accuracy: 0.7731
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4767 - accuracy: 0.8014
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4440 - accuracy: 0.8110
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4273 - accuracy: 0.8192
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4194 - accuracy: 0.8239
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4151 - accuracy: 0.8278
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4114 - accuracy: 0.8274
Epoch 8/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4080 - accuracy: 0.8304
Epoch 9/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4041 - accuracy: 0.8345
Epoch 10/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4001 - accura

In [13]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

ValueError: X has 12 features, but StandardScaler is expecting 13 features as input.

In [11]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [12]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1494  101]
 [ 181  224]]


0.859